In [1]:
import csv
import requests
import asyncio
import aiohttp

In [ ]:
import csv
import aiohttp
import asyncio

async def fetch_entity_info(session, url):
    async with session.post(lamAPI_url, headers=headers, json={"json": [url]}) as response:
        return await response.json()

async def process_row(session, row):
    urls = row[3].split(' ')
    for url in urls:
        parts = url.split('/')
        q_id = parts[-1]
        if q_id not in urls:
            urls.append(q_id)
            response = await fetch_entity_info(session, q_id)
            if response['wikidata']:
                for key, value in response['wikidata'][q_id]['objects'].items():
                    flag = True
                    if q_id[0] == 'P':
                        print(f"ID: {q_id} is a predicate {key} --> kind: PREDICATE")
                        predicate_list.append(q_id)
                        flag = False
                        break
                    if 'P31' in value:
                        if 'Q4167410' == key:
                            print(f"ID: {q_id} is instance of {key} --> kind: DISAMBIGUATION")
                            disambiguation_list.append(q_id)
                            flag = False
                            break
                        elif 'Q4167836' == key:
                            print(f"ID: {q_id} is instance of {key} --> kind: CATEGORY")
                            category_list.append(q_id)
                            flag = False
                            break
                    if 'P279' in value:
                        print(f"ID: {q_id} is subclass of {key} --> kind: TYPE")
                        type_list.append(q_id)
                        flag = False
                        break  
                if flag:
                    print(f"ID: {q_id} is an ENTITY")
                    entity_list.append(q_id)

async def main():
    csv_file_path = './my-data/Dataset/Dataset/Round1_T2D/gt/CEA_Round1_gt_WD.csv'
    async with aiohttp.ClientSession() as session:
        with open(csv_file_path, mode='r') as file_gt:
            csv_reader = csv.reader(file_gt)
            tasks = []
            for row in csv_reader:
                tasks.append(process_row(session, row))
            await asyncio.gather(*tasks)

# Define your lists outside of the asyncio scope
lamAPI_url = "https://lamapi.hel.sintef.cloud/entity/objects?token=lamapi_demo_2023&kg=wikidata&lang=en"
headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }


type_list = []
entity_list = []
predicate_list = []
disambiguation_list = []
category_list = []

# Run the asyncio loop
await main()


In [7]:
def chunks(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]


csv_file_path = './my-data/Dataset/Dataset/Round1_T2D/gt/CEA_Round1_gt_WD.csv'
urls = []

with open(csv_file_path, mode='r') as file_gt:
    # Create a CSV reader object
    csv_reader = csv.reader(file_gt)
    
    # Iterate over each row in the CSV file
    for row in csv_reader:        
        # Q_id extraction
        url = row[3].split(' ')
        for el in url:
            parts = el.split('/')
            el = parts[-1]
            if el not in urls:
                urls.append(el)
                
                            

In [11]:
!pip install backoff

In [ ]:
import asyncio
import aiohttp
import json
import backoff

@backoff.on_exception(backoff.expo,  # Use exponential backoff
                      (aiohttp.ClientError, aiohttp.http_exceptions.HttpProcessingError, asyncio.TimeoutError),
                      max_tries=5,  # Retry up to 5 times
                      max_time=300)  # Maximum total time to backoff is 300 seconds
async def fetch(session, url, data, headers, params, semaphore):
    async with semaphore:
        async with session.post(url, json=data, headers=headers, params=params) as response:
            response.raise_for_status()  # Raises an exception for 4XX/5XX status codes
            return await response.text()

async def main():
    url = "https://lamapi.hel.sintef.cloud/entity/objects"

    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    params = {
        "token": "lamapi_demo_2023",
        "kg": "wikidata"
    }
    semaphore = asyncio.Semaphore(50)  # Limit to 50 concurrent requests

    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=False)) as session:  
        tasks = []
        chunk_size = 500
        for chunk in chunks(urls, chunk_size):
            tasks.append(fetch(session, url, {"json": chunk}, headers, params, semaphore))
        responses = await asyncio.gather(*tasks, return_exceptions=True)
        for response in responses:
            if isinstance(response, Exception):
                print(f"Error: {response}")
            else:
                try:
                    result = json.loads(response)
                    print(result)  # You might want to print or process the result here
                except json.JSONDecodeError as e:
                    print(f"Failed to parse JSON response: {response}")

await main()  # Correct way to run the main coroutine



In [ ]:


    lamAPI_url = "https://lamapi.hel.sintef.cloud/entity/objects?token=lamapi_demo_2023&kg=wikidata&lang=en"

    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    type_list = []
    entity_list = []
    predicate_list = []
    disambiguation_list = []
    category_list = []


    data = {
        "json": [
            el
        ]
    }

    response = requests.post(lamAPI_url, headers=headers, json=data)
    if response.json()['wikidata']:
        matching_items = []
        for key, value in response.json()['wikidata'][el]['objects'].items():
            flag = True
            if el[0] == 'P':
                print(f"ID: {el} is a predicate {key} --> kind: PREDICATE")
                predicate_list.append(el)
                flag = False
                break
            if 'P31' in value:
                if 'Q4167410' == key:
                    print(f"ID: {el} is instance of {key} --> kind: DISAMBIGUATION")
                    disambiguation_list.append(el)
                    flag = False
                    break
                elif 'Q4167836' == key:
                    print(f"ID: {el} is instance of {key} --> kind: CATEGORY")
                    category_list.append(el)
                    flag = False
                    break
            if 'P279' in value:
                print(f"ID: {el} is subclass of {key} --> kind: TYPE")
                type_list.append(el)
                flag = False
                break  

        if flag:
            print(f"ID: {el} is an ENTITY")
            entity_list.append(el)
                